In [95]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP
using LinearAlgebra

include("2_4.jl")
inputfilepath = "data_2-4"
inputfilename = "Data_test_4_2_3.txt"

## Data
# nb_dem   : Nombre de demandeurs.
# nb_prod  : Nombre de produits
# nb_mag   : Nombre de magasins.
# nb_noeud : Nombre de noeuds.
# S        : Smp Stock de produit p dans le magasin M
# Q        : Qdp Quantité de produit p dans la commande d
# R        : Rij valeur de l'arc représentant la distance à parcourir/temps de trajet entre les sites i et j.
# N        : L'ensemble des noeuds.
nb_dem, nb_prod, nb_mag, nb_noeuds, S, Q, R = read_data_24(inputfilepath, inputfilename);

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:nb_mag, 1:nb_prod, 1:nb_dem] >= 0)
@variable(model, Z[1:(1+nb_dem),1:(1+nb_dem),1:nb_mag], Bin)

# define objective function
@objective(model, Min, sum(reshape(sum(Z[2:(1+nb_dem),2:(1+nb_dem),:],dims = 3),nb_dem,nb_dem).*reduce(hcat,R[1+nb_mag:nb_dem+nb_mag])[1+nb_mag:nb_dem+nb_mag,:])
                           +sum((Z[2:1+nb_dem,1,:] + Z[1,2:1+nb_dem,:]).*reduce(hcat,R[1:nb_mag])[1+nb_mag:nb_mag+nb_dem,:]))

#Contrainte2
for i in 1:nb_mag
    @constraint(model, sum(Z[1,2:1+nb_dem,i]) <= sum(M[i,:,:]))
    @constraint(model, sum(M[i,:,:]) /(1 + sum(S[i][:]))<= sum(Z[1,2:1+nb_dem,i]))
    @constraint(model, tr(Z[:,:,i])==0)
    for j in 1:1+nb_dem
        @constraint(model, sum(Z[j,:,i]) <= sum(M[i,:,:]))
        @constraint(model, sum(M[i,:,:])/(1+sum(S[i][:]))<= sum(Z[j,:,i]))
        @constraint(model, sum(Z[:,j,i]) <= sum(M[i,:,:]))
        @constraint(model, sum(M[i,:,:])/(1+sum(S[i][:]))<= sum(Z[:,j,i]))
    end
     for j in 1:nb_dem 
        @constraint(model, sum(Z[j,1:j,i])+sum(Z[1:j,j,i]) <= 1)
    end
    @constraint(model, sum(Z[2:1+nb_dem,1,i]) <= sum(M[i,:,:]))
    @constraint(model, sum(M[i,:,:]) /(1 + sum(S[i][:]))<= sum(Z[2:1+nb_dem,1,i]))
end

#Contrainte3
for i in 1:nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(M[:,j,i]) == Q[i][j])
    end
end

# Constrainte5:
for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model,sum(M[i,j,:]) <= S[i][j])
    end
end

# Z:
for i in 1:nb_dem
    for j in 1:nb_mag
        @constraint(model,sum(Z[i+1,:,j]) <= sum(M[j,:,i]))
        @constraint(model,sum(Z[:,i+1,j]) <= sum(M[j,:,i]))
        @constraint(model,sum(M[j,:,i])/(1+sum(Q[i][:])) <= sum(Z[:,i+1,j]))
        @constraint(model,sum(M[j,:,i])/(1+sum(Q[i][:])) <= sum(Z[i+1,:,j]))
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1749.13 - 0.00 seconds
Cgl0002I 10 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 83 rows, 51 columns (40 integer (40 of which binary)) and 865 elements
Cbc0012I Integer solution of 2971 found by DiveCoefficient after 3512 iterations and 0 nodes (0.38 seconds)
Cbc0031I 20 added rows had average density of 32.95
Cbc0013I At root node, 20 cuts changed objective from 1749.1304 to 2720.4183 in 100 passes
Cbc0014I Cut generator 0 (Probing) - 36 row cuts average 2.3 elements, 0 column cuts (0 active)  in 0.026 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 1570 row cuts average 49.8 el

In [98]:
JuMP.value.(Z)

5×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0